In [1]:
from torchvision.transforms import ToTensor, Resize, Compose, RandomAffine, ColorJitter
from PIL import Image
import numpy as np
import cv2
affine = RandomAffine(degrees=(-5, 5),
                    scale=(0.5, 1.05), shear=10)
jitter = ColorJitter(brightness=0.5, 
            contrast=0.5,
            saturation=0.5,
            hue=0.5)

image = Image.open(r'data\new_train\train_img_122.jpg').convert('L')
open_cv_image1 = np.array(image)
cv2.imshow('image', open_cv_image1)
image = affine(image)
image = jitter(image)
cv2.waitKey(0)

open_cv_image2 = np.array(image) 

cv2.imshow('image', open_cv_image2)
cv2.waitKey(0)



KeyboardInterrupt: 